In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_9.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,"Dear principal,\n\nI believe that students sho...",0.0
1,Many times we as humans study things that have...,0.0
2,"For Most, sucker is filled with unwinding from...",0.0
3,Some schools require students to complete summ...,0.0
4,Scientists believe that Venus could sustain hu...,0.0
...,...,...
17983,Driverless cars would Se good in the future Be...,0.0
17984,"In Vauban, Germany. Residents of the upscale c...",0.0
17985,I'm Even End E guy Et my work named John says ...,0.0
17986,No! The natural landform UAS not made by any k...,0.0


In [2]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 1900])
Attention Mask size: torch.Size([17988, 1900])
Vocabulary Size: 32800
Max Sequence Length: 1900


In [4]:
model=Sequential()
input_length=1900
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=100,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)



Epoch 1/5


450/450 [==============================] - 181s 399ms/step - loss: 0.0161 - accuracy: 0.9978
Epoch 2/5
450/450 [==============================] - 179s 398ms/step - loss: 4.7924e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 187s 417ms/step - loss: 2.4187e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 180s 399ms/step - loss: 1.4706e-05 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 179s 397ms/step - loss: 9.7870e-06 - accuracy: 1.0000


In [5]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)

113/113 [==============================] - 13s 109ms/step - loss: 8.0978e-06 - accuracy: 1.0000


[8.097787031147163e-06, 1.0]

In [6]:
model.save("newmodel9.keras")